In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import time

In [ ]:
file_path = '../data/train.csv'
df = pd.read_csv(file_path)
columns = '질문_1'

In [ ]:
target = df[columns]

In [ ]:
class any_text_except_specific_to_be_present_in_element_within_another_element:
    def __init__(self, container_locator, child_tag, excluded_text):
        self.container_locator = container_locator
        self.child_tag = child_tag
        self.excluded_text = excluded_text

    def __call__(self, driver):
        container_element = driver.find_element(*self.container_locator)
        child_elements = container_element.find_elements(By.TAG_NAME, self.child_tag)
        return any(child_element.text.strip() != "" and child_element.text.strip() != self.excluded_text for child_element in child_elements)

In [ ]:
new_data = []
driver = webdriver.Chrome()
driver.get('https://papago.naver.com/?sk=ko&tk=ja&hn=0')
for index, row in enumerate(tqdm(target)):
    # 사이트에 접속
    input_field = driver.find_element(By.ID, 'txtSource')
    input_field.send_keys(row)
    time.sleep(3)

    wait = WebDriverWait(driver, 10)
    target_locator = (By.ID, "targetEditArea")
    excluded_text = "..."  # 제외할 텍스트
    wait.until(any_text_except_specific_to_be_present_in_element_within_another_element(target_locator, "span", excluded_text))

    switch_button = driver.find_element(By.CLASS_NAME, "btn_switch___x4Tcl")
    switch_button.click()
    time.sleep(1)

    wait.until(any_text_except_specific_to_be_present_in_element_within_another_element(target_locator, "span", excluded_text))

    final_text = driver.find_element(By.ID, "targetEditArea").text
    new_data.append(final_text)
    time.sleep(1)
    input_field.clear()
    print(row, "<->",final_text)

# 데이터프레임에 새로운 열로 저장
df[f"{columns}'"] = new_data

In [ ]:
df = df[['id','질문_1',"질문_1'",'질문_2','category','답변_1','답변_2','답변_3','답변_4','답변_5']]

In [ ]:
df.to_csv("../data/train_aug.csv")